# Autoregressive (자동회귀) 문장 생성

- Colab 에서 실행

간단한 자동 회귀적인 텍스트 생성을 위한 코드 예제입니다. 여기서는 네이버의 HyperCLOVAX 모델을 사용합니다. HyperCLOVAX는 한국어에 특화된 대화형 언어 모델로, 시스템 메시지와 사용자 메시지를 구분하여 입력받는 구조를 가지고 있습니다. 이는 GPT 계열 모델과 유사한 자동회귀 구조를 가지고 있으므로, 동일한 접근 방식을 사용할 수 있습니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 문장 시작 부분
system_content = ""
user_content = "옛날 옛적에"
chat = [
    {"role": "user", "content": user_content},
]
inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_dict=True, return_tensors="pt")
inputs = inputs.to(model.device)

# 토큰화 결과 확인
print("전체 입력 토큰:", inputs['input_ids'])
print("토큰 개수:", inputs['input_ids'].shape[1])

# 전체 입력 토큰 디코딩
full_tokens = inputs['input_ids']

decoded_full = tokenizer.decode(full_tokens[0], skip_special_tokens=False)  # special_tokens도 포함해서 보기
print("전체 디코딩 결과 (special_tokens 포함):")
print(decoded_full)

# user_content 부분만 토큰화해서 확인
simple_tokens = tokenizer.encode(f"{user_content}", return_tensors="pt")
print(f"{user_content} 토큰:", simple_tokens)
print(f"{user_content} 토큰 개수:", simple_tokens.shape[1])

# 토큰을 다시 텍스트로 변환해서 확인
decoded = tokenizer.decode(simple_tokens[0], skip_special_tokens=True)
print("디코딩 결과:", decoded)

전체 입력 토큰: tensor([[100272,    882,    198,  36092,    249, 104013, 102476,  82068,  19954,
         100273,    198, 100272,  78191,    198]], device='cuda:0')
토큰 개수: 14
전체 디코딩 결과 (special_tokens 포함):
<|im_start|>user
옛날 옛적에<|im_end|>
<|im_start|>assistant

옛날 옛적에 토큰: tensor([[ 36092,    249, 104013, 102476,  82068,  19954]])
옛날 옛적에 토큰 개수: 6
디코딩 결과: 옛날 옛적에


In [ ]:
# 문장 생성
output_ids = model.generate(
    **inputs,    # inputs 딕셔너리를 언패킹해서 전달
    max_length=100,
    num_return_sequences=1,
    repetition_penalty=1.2,  # 반복 패널티
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
output_ids

tensor([[100272,    882,    198,  36092,    249, 104013, 102476,  82068,  19954,
         100273,    198, 100272,  78191,    198,      1,  36092,    249, 104013,
         102476,  82068,  19954,  21908,  16969, 109605, 104967, 102134, 103536,
          72043, 104562,     11, 107325, 106763,  57390,  61415, 106439, 107510,
         105077, 107468, 101632,     13,  23955, 107367, 105012, 104065,  81673,
         110211, 101721, 110288,     11, 104690, 108693,  54780, 109130,  21028,
          66610, 104685, 105873,  35495, 101632,    382, 106900, 110125,    330,
         108606, 107699,  66965,    498,    330, 104765, 106889,  66965,    498,
            330, 103339,  64189,  66965,      1, 106484, 109344,     11, 102388,
         105378, 104304, 103400,  81673, 101942,  18359, 105928, 107505, 101888,
         110125, 101074, 107097,  81673, 104284, 103049, 109686, 102520, 101632,
             13]], device='cuda:0')

In [ ]:
# 생성된 문장 출력 (CLOVAX 방식)
output_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]

# 필요시 <|endofturn|>, <|stop|> 등에서 자르기
for stop_str in ["<|endofturn|>", "<|stop|>"]:
    if stop_str in output_text:
        output_text = output_text.split(stop_str)[0]

print(f"Generated text: {output_text}")

Generated text: user
옛날 옛적에
assistant
"옛날 옛적에..."는 한국 전통 문학 작품 중 하나로, 주로 신화나 전설적인 이야기를 담고 있습니다. 이 작품은 다양한 문화와 시대를 배경으로 하며, 인간의 삶과 자연의 조화를 다루고 있습니다.

대표적으로 "춘향전", "심청전", "흥부전" 등이 있으며, 이들은 각각 다른 시대와 배경을 가지고 있지만 공통적으로 인간 관계와 도덕성을 강조하고 있습니다.


HyperCLOVAX는 자체적으로 autoregressive 모델입니다. "Autoregressive"란, 이전에 생성된 토큰들을 기반으로 다음 토큰을 생성하는 모델을 의미합니다.

위의 코드에서 `model.generate` 메서드는 이미 autoregressive한 방식으로 문장을 생성합니다. 그러나 이를 명시적으로 보여주기 위해 각 단계에서 토큰을 하나씩 생성하는 autoregressive한 코드를 아래에 작성하겠습니다:

In [ ]:
# 문장 시작 부분
user_content = "옛날 옛적에"
chat = [
    {"role": "user", "content": user_content},
]
inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_dict=True, return_tensors="pt")
inputs = inputs.to(model.device)
inputs

{'input_ids': tensor([[100272,    882,    198,  36092,    249, 104013, 102476,  82068,  19954,
         100273,    198, 100272,  78191,    198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
# 모델 추론
predictions = model(**inputs)  # inputs 딕셔너리를 언패킹해서 전달
logits = predictions.logits
print(logits.shape)
logits

torch.Size([1, 14, 110592])


tensor([[[-1.4064, -0.9157, -0.1506,  ..., -1.1698, -1.1692, -1.1691],
         [ 9.2706, 12.9401, 11.9009,  ...,  3.7357,  3.7357,  3.7366],
         [ 6.2054,  9.7094, 10.5673,  ..., -0.0378, -0.0385, -0.0376],
         ...,
         [ 3.6117,  2.1378,  9.5609,  ...,  0.8532,  0.8533,  0.8542],
         [10.9762,  9.0058,  9.4168,  ...,  3.4993,  3.4983,  3.4996],
         [ 8.0550, 19.3332, 13.4940,  ...,  1.6563,  1.6563,  1.6562]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [ ]:
# Autoregressive한 방식으로 문장 생성
max_length = 50
input_ids_concat = inputs['input_ids'].clone()  # inputs에서 input_ids 추출

while input_ids_concat.shape[1] < max_length:
    # 다음 토큰 예측
    model_inputs = {'input_ids': input_ids_concat}
    if 'attention_mask' in inputs:
        model_inputs['attention_mask'] = torch.ones_like(input_ids_concat)

    predictions = model(**model_inputs)
    logits = predictions.logits
    predicted_token = torch.argmax(logits[0, -1]).item()
    #print(predicted_token)

    # 생성된 토큰을 입력 토큰 뒤에 추가
    input_ids_concat = torch.cat([input_ids_concat, torch.tensor([[predicted_token]], device=input_ids_concat.device)], dim=1)
    print(input_ids_concat)

tensor([[100272,    882,    198,  36092,    249, 104013, 102476,  82068,  19954,
         100273,    198, 100272,  78191,    198,  36092]], device='cuda:0')
tensor([[100272,    882,    198,  36092,    249, 104013, 102476,  82068,  19954,
         100273,    198, 100272,  78191,    198,  36092,    249]],
       device='cuda:0')
tensor([[100272,    882,    198,  36092,    249, 104013, 102476,  82068,  19954,
         100273,    198, 100272,  78191,    198,  36092,    249, 104013]],
       device='cuda:0')
tensor([[100272,    882,    198,  36092,    249, 104013, 102476,  82068,  19954,
         100273,    198, 100272,  78191,    198,  36092,    249, 104013, 102476]],
       device='cuda:0')
tensor([[100272,    882,    198,  36092,    249, 104013, 102476,  82068,  19954,
         100273,    198, 100272,  78191,    198,  36092,    249, 104013, 102476,
          82068]], device='cuda:0')
tensor([[100272,    882,    198,  36092,    249, 104013, 102476,  82068,  19954,
         100273,    198,

In [ ]:
# 생성된 문장 출력
decoded_text = tokenizer.decode(input_ids_concat[0], skip_special_tokens=True)

# 필요시 <|endofturn|>, <|stop|> 등에서 자르기
for stop_str in ["<|endofturn|>", "<|stop|>"]:
    if stop_str in decoded_text:
        decoded_text = decoded_text.split(stop_str)[0]

print(decoded_text)

user
옛날 옛적에
assistant
옛날 옛적에, 어느 마을에 한 소년이 살고 있었습니다. 그는 마을의 중심에 있는 큰 나무 아래에서 책을 읽고 있었습니다. 그
